In [8]:
import pandas as pd
import numpy as np
import glob

### Carregando as bases de dados manipuladas n2

In [19]:
df_carga : pd.DataFrame = pd.read_csv('./data/checkpoints/df_carga_v1.csv',sep=	';')
df_clima : pd.DataFrame = pd.read_csv('./data/checkpoints/df_clima_grouped_mean.csv')

print(df_carga.shape, df_clima.shape)

(89326, 10) (87672, 9)


In [24]:
df_carga.head(2)

,Unnamed: 0,id_subsistema,nom_subsistema,din_instante,val_cargaenergiahomwmed,year,month,day,hour,date
0,3,SE,SUDESTE,2012-01-01 00:00:00,28212.46,2012,1,1,0,2012-01-01
1,7,SE,SUDESTE,2012-01-01 01:00:00,27511.77,2012,1,1,1,2012-01-01


In [21]:
df_clima.head(2)


,data-hora,precipitacao_total,pressao_atmosferica,radiacao_global,temperatura_ar,temperatura_ponto_carvalho,umidade_relativa_ar,vento_direcao,velocidade_vento
0,2012-01-01 00:00:00,0.88,947.22,-2.8518,21.70,19.7,90.25,95.8,1.48
1,2012-01-01 01:00:00,2.00,947.30,-2.4346,21.24,19.5,92.00,114.0,0.80


In [25]:
# Transformando as colunas de data-hora em index
df_carga : pd.DataFrame = df_carga.set_index('din_instante')
df_clima : pd.DataFrame = df_clima.set_index('data-hora')

In [31]:
def find_row_by_index(df : pd.DataFrame, date : str,) -> pd.DataFrame:
    """
    Função para encontrar uma linha de um dataframe a partir de um index
    
    Parameters
    ----------
    df : pd.DataFrame
        Dataframe a ser pesquisado
    date : str
        Index a ser pesquisado
    Returns
    -------
    pd.DataFrame
        Dataframe com a linha pesquisada
    """
    return df.filter(items=[date], axis=0)

In [33]:
find_row_by_index(df_carga, '2021-12-31 22:00:00') # Filtrando a data especifica

,Unnamed: 0,id_subsistema,nom_subsistema,val_cargaenergiahomwmed,year,month,day,hour,date
din_instante,,,,,,,,,
2021-12-31 22:00:00,350435,SE,SUDESTE,35492.978,2021,12,31,22,2021-12-31


In [40]:
## Unindo as bases de dados
df_full : pd.DataFrame = pd.merge(df_carga, df_clima, left_index=True, right_index=True, how='inner')
# df_full.head()

df_full_v1 : pd.DataFrame = df_full[['precipitacao_total','pressao_atmosferica',
    'radiacao_global','temperatura_ar','temperatura_ponto_carvalho','umidade_relativa_ar','vento_direcao','velocidade_vento',
    'id_subsistema','nom_subsistema','val_cargaenergiahomwmed'
    ]]

df_full_v1.to_csv("./data/checkpoints/df_full_v1.csv")
print(df_full_v1.shape)
df_full_v1.head(3)


(87670, 11)


,precipitacao_total,pressao_atmosferica,radiacao_global,temperatura_ar,temperatura_ponto_carvalho,umidade_relativa_ar,vento_direcao,velocidade_vento,id_subsistema,nom_subsistema,val_cargaenergiahomwmed
2012-01-01 00:00:00,0.88,947.22,-2.8518,21.70,19.700,90.25,95.8,1.48,SE,SUDESTE,28212.46
2012-01-01 01:00:00,2.00,947.30,-2.4346,21.24,19.500,92.00,114.0,0.80,SE,SUDESTE,27511.77
2012-01-01 02:00:00,1.24,947.00,-2.3902,21.38,19.775,92.25,159.2,0.62,SE,SUDESTE,26339.97


In [39]:
find_row_by_index(df_full_v1, '2021-12-31 22:00:00')

,precipitacao_total,pressao_atmosferica,radiacao_global,temperatura_ar,temperatura_ponto_carvalho,umidade_relativa_ar,vento_direcao,velocidade_vento,id_subsistema,nom_subsistema,val_cargaenergiahomwmed
2021-12-31 22:00:00,0.12,938.78,131.3264,22.16,19.44,85.4,145.4,1.38,SE,SUDESTE,35492.978


In [43]:
## Checando os valores mínimos e máximos de data
df_full_v1.index.min(), df_full_v1.index.max()

('2012-01-01 00:00:00', '2021-12-31 23:00:00')